# Bibliotecas


In [1]:
import sys, os, logging, ee, folium, glob, rasterio
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import tensorflow as tf
import json, datetime, math
import numpy
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
from rasterio.plot import show
from osgeo import ogr, gdal



logging.getLogger('googleapicliet.discovery_cache').setLevel(logging.ERROR)
GPU_AFFINTY  = 0
GPU_MEMORY_LIMIT_GB = 10

gpu_dict = {'4090':{'GPU_AFFINTY' : 0, 'GPU_MEMORY_LIMIT_GB':12}, 
            '2070':{'GPU_AFFINTY':1, 'GPU_MEMORY_LIMIT_GB':8}}

sel_gpu = '4090'
GPU_AFFINTY  = gpu_dict[sel_gpu]['GPU_AFFINTY']
GPU_MEMORY_LIMIT_GB =gpu_dict[sel_gpu]['GPU_MEMORY_LIMIT_GB']


try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

# GPU config

In [2]:
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'

print('Tensorflow Version:',tf.__version__)
print('Folium Version:',folium.__version__)

gpus = tf.config.list_physical_devices('GPU')
print(tf.config.list_physical_devices('GPU'))
if gpus:
  try:
    tf.config.set_visible_devices(gpus[GPU_AFFINTY], 'GPU')
    GPU_MEMORY_LIMIT_GB = GPU_MEMORY_LIMIT_GB * 1e3
    # Currently, memory growth needs to be the same across GPUs
    if GPU_MEMORY_LIMIT_GB == 0:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    else:
        tf.config.set_logical_device_configuration(gpus[GPU_AFFINTY],[tf.config.LogicalDeviceConfiguration(memory_limit=GPU_MEMORY_LIMIT_GB)])
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

Tensorflow Version: 2.15.0
Folium Version: 0.14.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
1 Physical GPUs, 1 Logical GPUs


In [3]:
def creatDirectory(new_folder):
    if not os.path.exists(new_folder):
        print(f'lets make the directory: {new_folder}')
        os.makedirs(new_folder)
    else: return

# ENV Configs

In [4]:
# General 
GPU_AFFINITY   = gpus[GPU_AFFINTY].name
VERSION        = 'DATA_AUGMENTATION'
# VERSION        = '4_p5_WO_DATA_AUGMENTATION'
MOSAIC_VERSION = '1'
GOAL_CLASS     = 'apicum'
GOAL_YEAR      = '2022'

FOLDER_TRAIN   = 'training_samples'
FOLDER_EVAL    = 'eval_samples'
TRAINING_BASE  = 'training_patches_'+ MOSAIC_VERSION
EVAL_BASE      = 'eval_patches_'+ MOSAIC_VERSION

#Local paths
LOCAL_PATH  = '~/local_path/apicum_segmentation'
MODEL_DIR   = LOCAL_PATH+'/checkpoint/v'+VERSION
OUTPUT_PATH = LOCAL_PATH+'/output/v'+VERSION
creatDirectory(MODEL_DIR)
creatDirectory(OUTPUT_PATH)

# Exportation Configs
FOLDER_patch = 'allPatch'

# Specify inputs (Landsat bands) to the model and the response variable.
opticalBands = ['swir1', 'nir', 'red','green','MNDWI','NDVI'] 

# FOR DEEPLABV3
# opticalBands = ['red','green','blue'] #['swir1', 'nir', 'red','NDVI','MNDWI'] DEFAULT
BANDS        = opticalBands
RESPONSE     = 'supervised'
FEATURES     = BANDS + [RESPONSE]

# Specify the size and shape of patches expected by the model.
KERNEL_SIZE = 256
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]
COLUMNS = [
  tf.io.FixedLenFeature(shape=KERNEL_SHAPE, dtype=tf.float32) for k in FEATURES
]
FEATURES_DICT = dict(zip(FEATURES, COLUMNS))

# Sizes of the training and evaluation datasets.
TRAIN_SIZE = 0
EVAL_SIZE  = 0

# Specify model training parameters.
BATCH_SIZE  = 10 
DROPOUT     = 0.3
EPOCHS      = 50
BUFFER_SIZE = 1000 
OPTIMIZER   = 'Nadam' 
LOSS        = 'BinaryCrossentropy'
METRICS     = ['RootMeanSquaredError']

['swir1', 'nir', 'red', 'green', 'MNDWI', 'NDVI']


# Data Visualization

In [ ]:
baseClassV       = '4'
yearClass_class  = '2022'
yearClass_mosaic = '2022'
version_final    = '4'
classID          = 32

supervised_layer  = ee.Image('projects/apicum-segmentation/assets/supervisedImage_unet_apicum_2022_final').eq(classID).rename(RESPONSE);
supervisedChannel = supervised_layer.toByte().rename(RESPONSE);

'''  DEFAULT '''
image = ee.Image('projects/mapbiomas-workspace/TRANSVERSAIS/ZONACOSTEIRA6/mosaic_'+yearClass_mosaic).addBands(supervisedChannel)
mapid = image.getMapId({'bands': ['swir1', 'nir', 'red'], 'min': 30, 'max': 150})

# '''  FOR DEEPLABV3 '''
# image = ee.Image('projects/mapbiomas-workspace/TRANSVERSAIS/ZONACOSTEIRA6/mosaic_'+yearClass_mosaic+'_with_blue').addBands(supervisedChannel)
# mapid = image.getMapId({'bands': ['red', 'green', 'blue'], 'min': 11, 'max': 95})

map = folium.Map(location=[-23.0089, -43.6078],zoom_start=13)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Planet',
    overlay=True,
    name='Mosaic composite',
  ).add_to(map)
mapid = supervisedChannel.select(RESPONSE).mask(supervisedChannel.eq(1)).getMapId({'min': 0, 'max': 1, 'palette':'red'})
# map = folium.Map(location=[-23.0089, -43.6078])
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    overlay=True,
    name='Apicum '+yearClass_class,
  ).add_to(map)
map.add_child(folium.LayerControl())
map

In [46]:
featureStack = ee.Image.cat([
  image.select(BANDS).unmask(0),
  image.select(RESPONSE).unmask(0)
]).float()

list = ee.List.repeat(1, KERNEL_SIZE)
lists = ee.List.repeat(list, KERNEL_SIZE)
kernel = ee.Kernel.fixed(KERNEL_SIZE, KERNEL_SIZE, lists)

arrays = featureStack.neighborhoodToArray(kernel)

In [ ]:
yearClass_geoms = '2020'
trainingPolys_v1 = ee.FeatureCollection('projects/apicum-segmentation/assets/Geometries/trainPolys_apicum_p1')
evalPolys_v1     = ee.FeatureCollection('projects/apicum-segmentation/assets/Geometries/testPolys_apicum_p1')

trainingPolys_v2 = ee.FeatureCollection('projects/apicum-segmentation/assets/Geometries/trainPolys_apicum_p2')
evalPolys_v2     = ee.FeatureCollection('projects/apicum-segmentation/assets/Geometries/testPolys_apicum_p2')

trainingPolys_v3 = ee.FeatureCollection('projects/apicum-segmentation/assets/Geometries/trainPolys_apicum_p3')
evalPolys_v3     = ee.FeatureCollection('projects/apicum-segmentation/assets/Geometries/testPolys_apicum_p3')

trainingPolys = trainingPolys_v1.merge(trainingPolys_v2)
evalPolys     = evalPolys_v1.merge(evalPolys_v2).merge(evalPolys_v3)

id_filter_out = ['2_00000000000000000018','2_0000000000000000001f','1_00000000000000000004','2_0000000000000000001d']
trainingPolys = trainingPolys.filter(ee.Filter.inList('system:index', id_filter_out).Not())

trainingPolys = trainingPolys.merge(trainingPolys_v3)

def geo_type(feature):
    return feature.set('geo_type', feature.geometry().type())
trainingPolys = trainingPolys.map(lambda feat: geo_type(feat))
trainingPolys = trainingPolys.filter(ee.Filter.neq('geo_type','LineString'))

print(trainingPolys.size().getInfo())
print(evalPolys.size().getInfo())

polyImage = ee.Image(0).byte().paint(trainingPolys, 1).paint(evalPolys, 2)
polyImage = polyImage.updateMask(polyImage)

mapid = polyImage.getMapId({'min': 1, 'max': 2, 'palette': ['red', 'blue']})
map = folium.Map(location=[-1.3621, -45.2738], zoom_start=5)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    overlay=True,
    name='training polygons',
  ).add_to(map)
map.add_child(folium.LayerControl())
# map

# Train/Validation Sampling Exportation

In [50]:
# Convert the feature collections to lists for iteration.
trainingPolysList = trainingPolys.toList(trainingPolys.size())
evalPolysList = evalPolys.toList(evalPolys.size())
# These numbers determined experimentally.
n = 20 # Number of shards in each polygon.
N = 200 # Total sample size in each polygon.

version_samples_acc = "final"

#Add some generalism
TRAIN_SIZE = trainingPolys.size().getInfo()*N
EVAL_SIZE = evalPolys.size().getInfo()*N
print('TRAIN:'+str(TRAIN_SIZE))
print('EVAL:'+str(EVAL_SIZE))
GDRIVE = 'Apicum_segmentation'
# Export all the training data (in many pieces), with one task 
# per geometry.
for g in range(trainingPolys.size().getInfo()):
  geomSample = ee.FeatureCollection([])
  for i in range(n):
    sample = arrays.sample(
      region = ee.Feature(trainingPolysList.get(g)).geometry(), 
      scale = 30, 
      numPixels = N / n, # Size of the shard.
      seed = i,
      tileScale = 8
    )
    geomSample = geomSample.merge(sample)
  
  desc = TRAINING_BASE + '_g' + str(g)
  task = ee.batch.Export.table.toDrive(
    collection = geomSample,
    description = desc, 
    folder = GDRIVE+'/'+FOLDER_TRAIN+'_v'+version_samples_acc, 
    fileNamePrefix = desc,
    fileFormat = 'TFRecord',
    selectors = BANDS + [RESPONSE]
  )
  task.start()

# Export all the evaluation data.
for g in range(evalPolys.size().getInfo()):
  geomSample = ee.FeatureCollection([])
  for i in range(n):
    sample = arrays.sample(
      region = ee.Feature(evalPolysList.get(g)).geometry(), 
      scale = 30, 
      numPixels = N / n,
      seed = i,
      tileScale = 8
    )
    geomSample = geomSample.merge(sample)
  
  desc = EVAL_BASE + '_g' + str(g)
  task = ee.batch.Export.table.toDrive(
    collection = geomSample,
    description = desc, 
    folder = GDRIVE+'/'+FOLDER_EVAL+version_samples_acc, 
    fileNamePrefix = desc,
    fileFormat = 'TFRecord',
    selectors = BANDS + [RESPONSE],
  )
  task.start()

TRAIN:31800
EVAL:13400


# Datasets auxiliary reading functions

In [6]:
def parse_tfrecord(example_proto):
  """The parsing function.
  Read a serialized example into the structure defined by FEATURES_DICT.
  Args:
    example_proto: a serialized Example.
  Returns: 
    A dictionary of tensors, keyed by feature name.
  """
  print(FEATURES_DICT)
  return tf.io.parse_single_example(example_proto, FEATURES_DICT)



def to_tuple(inputs):
  """Function to convert a dictionary of tensors to a tuple of (inputs, outputs).
  Turn the tensors returned by parse_tfrecord into a stack in HWC shape.
  Args:
    inputs: A dictionary of tensors, keyed by feature name.
  Returns: 
    A dtuple of (inputs, outputs).
  """
  inputsList = [inputs.get(key) for key in FEATURES]
  stacked = tf.stack(inputsList, axis=0)
  # Convert from CHW to HWC
  stacked = tf.transpose(stacked, [1, 2, 0])
  return stacked[:,:,:len(BANDS)], stacked[:,:,len(BANDS):]


def get_dataset(pattern):
  """Function to read, parse and format to tuple a set of input tfrecord files.
  Get all the files matching the pattern, parse and convert to tuple.
  Args:
    pattern: A file pattern to match in a Cloud Storage bucket.
  Returns: 
    A tf.data.Dataset
  """
  # glob = tf.gfile.Glob(pattern) for tendorflow 1.x
  glob = tf.io.gfile.glob(pattern) # for tendorflow 2.x
  dataset = tf.data.TFRecordDataset(glob, compression_type='GZIP')
  dataset = dataset.map(parse_tfrecord, num_parallel_calls=5)
  dataset = dataset.map(to_tuple, num_parallel_calls=5)
  return dataset

# Read and mount train/validation datasets

In [6]:
from tensorflow.keras import layers
import copy
import random
seed = 42
tf.random.set_seed(seed)

data_augmentation_model = tf.keras.Sequential([
    layers.RandomFlip("vertical", seed=seed),
    layers.RandomRotation(factor=0.2, seed=seed),
    layers.RandomTranslation(height_factor=0.1, width_factor=0.1, seed=seed)
])

def create_data_augmentation_v2(model):

    return model

def augment_fn(x, y):
    # data_augmentation_func = create_data_augmentation(seed)
    data_augmentation_func = create_data_augmentation_v2(data_augmentation_model)
    input_data = tf.concat([x, y], axis=-1)
    
    # Check if the instance contains your target class
    tensor_sum_value  = tf.math.reduce_sum(y, axis=[0,1,2])
    tensor_base_value = tf.constant([183.], dtype=tf.float32)
    greater_tensor    = tf.greater(tensor_sum_value, tensor_base_value)

    if tf.reduce_any(greater_tensor):
        augmented_data = data_augmentation_func(input_data)
        x_aug = augmented_data[:, :, :len(BANDS)]
        y_aug = augmented_data[:, :, len(BANDS):]
        return (x_aug, y_aug)
    else:
        return (x, y)
    
def is_augmented(x, y):
    tensor_sum_value  = tf.math.reduce_sum(y, axis=[0,1,2])
    tensor_base_value = tf.constant([183.], dtype=tf.float32)
    greater_tensor    = tf.greater(tensor_sum_value, tensor_base_value)
    return tf.reduce_any(greater_tensor)


def get_training_dataset_data_aug_target(SAMPLE_PATH):
    """Get the preprocessed training dataset
  Returns: 
    A tf.data.Dataset of training data.
  """
    
    glob    = SAMPLE_PATH + '/'+ TRAINING_BASE + '*'   
    dataset_bkp = get_dataset(glob)
    dataset     = get_dataset(glob)
    print('\n')
    
    
    num_samples = 31800
    
    for index in range(1):
        augmented_dataset = dataset_bkp.map(augment_fn)
        augmented_only_dataset = augmented_dataset.filter(is_augmented).take(15900)
        dataset = dataset.concatenate(augmented_only_dataset)
    
    # dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
    # train_da_size = dataset.reduce(np.int64(0), lambda x,_ : x + 1).numpy()
    # print(train_da_size) #46767
    # 250 => 46767
    # 183 => 47701
    # 182 => 47732
    # 180 => 47750
    # 175 => 47854
    # 170 => 47931
    dataset = dataset.shuffle(BUFFER_SIZE, reshuffle_each_iteration=True).batch(12).repeat().prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset

TRAIN_PATH = '~/local_path/apicum_segmentation/train/samples_v'+version_samples_acc
creatDirectory(TRAIN_PATH)
training = get_training_dataset_data_aug_target(TRAIN_PATH)

{'swir1': FixedLenFeature(shape=[256, 256], dtype=tf.float32, default_value=None), 'nir': FixedLenFeature(shape=[256, 256], dtype=tf.float32, default_value=None), 'red': FixedLenFeature(shape=[256, 256], dtype=tf.float32, default_value=None), 'green': FixedLenFeature(shape=[256, 256], dtype=tf.float32, default_value=None), 'MNDWI': FixedLenFeature(shape=[256, 256], dtype=tf.float32, default_value=None), 'NDVI': FixedLenFeature(shape=[256, 256], dtype=tf.float32, default_value=None), 'supervised': FixedLenFeature(shape=[256, 256], dtype=tf.float32, default_value=None)}
{'swir1': FixedLenFeature(shape=[256, 256], dtype=tf.float32, default_value=None), 'nir': FixedLenFeature(shape=[256, 256], dtype=tf.float32, default_value=None), 'red': FixedLenFeature(shape=[256, 256], dtype=tf.float32, default_value=None), 'green': FixedLenFeature(shape=[256, 256], dtype=tf.float32, default_value=None), 'MNDWI': FixedLenFeature(shape=[256, 256], dtype=tf.float32, default_value=None), 'NDVI': FixedLenFe

In [ ]:
def get_eval_dataset(SAMPLE_PATH):
    glob    = SAMPLE_PATH + '/'+ EVAL_BASE + '*'
    dataset = get_dataset(glob)
    # eval_size = dataset.reduce(np.int64(0), lambda x,_ : x + 1).numpy()
    # print(eval_size)  #13355
    dataset = dataset.batch(1).repeat()  
    # return dataset, eval_size
    return dataset

EVAL_PATH = '~/local_path/apicum_segmentation/eval/samples_v'+version_samples_acc
creatDirectory(EVAL_PATH)
evaluation = get_eval_dataset(EVAL_PATH)

# U-shaped model

In [8]:
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import models
from tensorflow.keras import metrics
from tensorflow.keras import optimizers


def conv_block(input_tensor, num_filters):
    encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(input_tensor)
    encoder = layers.BatchNormalization()(encoder)
    encoder = layers.Activation('relu')(encoder)
    encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(encoder)
    encoder = layers.BatchNormalization()(encoder)
    encoder = layers.Activation('relu')(encoder)
    return encoder

def encoder_block(input_tensor, num_filters):
    encoder = conv_block(input_tensor, num_filters)
    encoder_pool = layers.MaxPooling2D((2, 2), strides=(2, 2))(encoder)
    return encoder_pool, encoder

def decoder_block(input_tensor, concat_tensor, num_filters):
    decoder = layers.Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(input_tensor)
    decoder = layers.concatenate([concat_tensor, decoder], axis=-1)
    decoder = layers.BatchNormalization()(decoder)
    decoder = layers.Activation('relu')(decoder)
    decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
    decoder = layers.BatchNormalization()(decoder)
    decoder = layers.Activation('relu')(decoder)
    decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
    decoder = layers.BatchNormalization()(decoder)
    decoder = layers.Activation('relu')(decoder)
    return decoder

def get_model():
    inputs = layers.Input(shape=[None, None, len(BANDS)]) # 256
    encoder0_pool, encoder0 = encoder_block(inputs, 64) # 128
    encoder1_pool, encoder1 = encoder_block(encoder0_pool, 128) # 64
    encoder2_pool, encoder2 = encoder_block(encoder1_pool, 256) # 32
    encoder3_pool, encoder3 = encoder_block(encoder2_pool, 512) # 16
    center = conv_block(encoder3_pool, 1024) # 8 center
    decoder4 = decoder_block(center, encoder3, 512) # 16
    decoder3 = decoder_block(decoder4, encoder2, 256) # 32
    decoder2 = decoder_block(decoder3, encoder1, 128) # 64
    decoder1 = decoder_block(decoder2, encoder0, 64) # 128
    dropout = layers.Dropout(DROPOUT, name="dropout", noise_shape=None, seed=None)(decoder1)
    outputs = layers.Conv2D(1, (1, 1),  activation=tf.nn.sigmoid, padding='same', kernel_initializer=tf.keras.initializers.GlorotNormal())(dropout)
    
    model = models.Model(inputs=[inputs], outputs=[outputs])
    optimizer = tf.keras.optimizers.Nadam(0.000005, name='optimizer')# LR DEFAULT
    # optimizer = tf.keras.optimizers.Nadam(0.00001, name='optimizer')# LR 1e-5
    
    
    model.compile(
        optimizer=optimizer, 
        loss=losses.get(LOSS),
        metrics=[metrics.get(metric) for metric in METRICS]
    )

    return model

In [ ]:
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt

def previewClass(epoch,log):
    counter = 0
    for batch in evaluation.shuffle(1000).take(3):
        pureImage = batch[0]
        supervised = batch[1]
        stacked = tf.transpose(pureImage[0], [0, 1, 2]).numpy()
        stackedS = tf.transpose(supervised[0], [0, 1, 2]).numpy()
        test_pred_raw = m.predict(pureImage)
        test_pred_raw = tf.transpose(test_pred_raw[0],[0, 1, 2]).numpy()
        fig = plt.figure(figsize=[12,4])
        # show original image
        fig.add_subplot(131)
        plt.imshow(stacked[:,:,0:3].astype(np.uint8), interpolation='nearest', vmin=0, vmax=255)
        fig.add_subplot(132)
        plt.imshow(stackedS[:,:,0], interpolation='nearest',cmap="gray")
        fig.add_subplot(133)
        plt.imshow(test_pred_raw[:,:,0], interpolation='nearest',cmap="gray")
        plt.show()
        counter = counter+1
# previewClass(1,1)  

# Multiclass semantic segmentation using DeepLabV3+

In [63]:
''' 
https://keras.io/examples/vision/deeplabv3_plus/
https://arxiv.org/pdf/1802.02611.pdf

https://keras.io/examples/vision/deeplabv3_plus/ AND https://colab.research.google.com/github/keras-team/keras-io/blob/master/examples/vision/ipynb/deeplabv3_plus.ipynb
https://github.com/lattice-ai/DeepLabV3-Plus/tree/master

DeepLabv3+ extends DeepLabv3 by adding an encoder-decoder structure. 
The encoder module processes multiscale contextual information by applying dilated convolution at multiple scales, 
while the decoder module refines the segmentation results along object boundaries.

Deep Lab V3 + extends the architecture of Deep Lab V3 and Deep Lab V2. It utilises the power of Atrous Convolution and Spatial Pyramid Pooling.

It combines the Atrous Convolution and Spatial Pyramid Pooling(ASPP) 
and forms the Atrous Spatial Pyramid Pooling. Which allows to extract features at multiple scales. 
Instead of using a cascade or a parallel method, it uses an Encoder-Decoder architecture.

In which the Encoder learns from the high level features of ResNet50 using ASPP, Lower Level Features(LLF) of ResNet 50 are also extracted. 
This is because the LLF and the High Level features encodes different information and 
the geometric information about the object is extracted from the LLF, whereas the information about the object class is extracted from the HLF.

ResNet-101 [25] as network backbone in the proposed DeepLabv3+ model




Deep lab v3 with water bodie examples:  https://www.kaggle.com/code/utkarshsaxenadn/water-bodies-segmentation-deeplabv3#Data-Loading

'''
import keras
from tensorflow.keras import layers
from tensorflow.keras import losses, metrics, optimizers

from keras.layers import Input

# Pretrained Model
from tensorflow.keras.applications import ResNet50


def conv_block_dlv3(input_tensor, num_filters=256, kernel_size=3, dilation_rate=1, padding="same", use_bias=False):
    encoder = layers.Conv2D(
                num_filters,
                kernel_size        = kernel_size,
                dilation_rate      = dilation_rate,
                padding            = "same",
                use_bias           = use_bias,
                kernel_initializer = keras.initializers.HeNormal())(input_tensor)
    encoder = layers.BatchNormalization()(encoder)
    encoder = layers.Activation('relu')(encoder)
    return encoder

def DilatedSpatialPyramidPooling_dlv3(dspp_input):
    dims = dspp_input.shape #  B, H, W, C
    print(dims)
    
    # Image Pooling
    x    = layers.AveragePooling2D(pool_size=(dims[-3], dims[-2]))(dspp_input) # dims[-3] == height(H) and dims[-2](W) width
    x    = conv_block_dlv3(x, kernel_size=1, use_bias=True)
    out_pool = layers.UpSampling2D(
        size =(dims[-3] // x.shape[1], dims[-2] // x.shape[2]), interpolation="bilinear", name="ASPP-ImagePool-UpSample",
    )(x)

    # Atrous Oprtations
    out_1  = conv_block_dlv3(dspp_input, kernel_size=1, dilation_rate=1)
    out_6  = conv_block_dlv3(dspp_input, kernel_size=3, dilation_rate=6)
    out_12 = conv_block_dlv3(dspp_input, kernel_size=3, dilation_rate=12)
    out_18 = conv_block_dlv3(dspp_input, kernel_size=3, dilation_rate=18)

    # Combine All
    x      = layers.Concatenate(axis=-1, name="ASPP-Combine")([out_pool, out_1, out_6, out_12, out_18])
    output = conv_block_dlv3(x, kernel_size=1)
    
    # Final Output
    return output

def DeeplabV3Plus(IMAGE_SIZE, num_classes):
    ''' ENCODER PHASE '''
    # Input
    # model_input = keras.Input(shape=(image_size, image_size, 3)) # FROM  https://keras.io/examples/vision/deeplabv3_plus/ AND https://colab.research.google.com/github/keras-team/keras-io/blob/master/examples/vision/ipynb/deeplabv3_plus.ipynb
    model_input = Input(shape=[IMAGE_SIZE, IMAGE_SIZE, len(BANDS)]) # 256 (shape=[256, 256, len(BANDS)])
    
    # Base Mode
    resnet50 = keras.applications.ResNet50(
        weights = "imagenet", include_top=False, input_tensor=model_input
    )
    
    # Atrous Spatial Pyramid Pooling(ASPP) Phase
    DCNN = resnet50.get_layer("conv4_block6_2_relu").output
    ASPP = DilatedSpatialPyramidPooling_dlv3(DCNN)

    
    ''' DECODER PHASE '''
    # Output from ASPP(ENCODER PHASE) with Upsample by 4 
    ASPP = layers.UpSampling2D(
        size = (IMAGE_SIZE // 4 // ASPP.shape[1], IMAGE_SIZE // 4 // ASPP.shape[2]),
        interpolation="bilinear",
    )(ASPP)
    
    # Lower Level Features(LLF) Phase
    LLF = resnet50.get_layer("conv2_block3_2_relu").output
    LLF = conv_block_dlv3(LLF, num_filters=48, kernel_size=1)

    # Combined
    combined = layers.Concatenate(axis=-1)([ASPP, LLF])
    features = conv_block_dlv3(combined)
    features = conv_block_dlv3(features)
    
    # Upsample by 4
    upsample = layers.UpSampling2D(
        size=(IMAGE_SIZE // features.shape[1], IMAGE_SIZE // features.shape[2]),
        interpolation="bilinear",
    )(features)
    
    # Output Mask
    model_output = layers.Conv2D(num_classes, kernel_size=(1, 1), padding="same")(upsample)
    # return keras.Model(inputs=model_input, outputs=model_output)
    
    model = keras.Model(inputs=model_input, outputs=model_output)
    
    optimizer = tf.keras.optimizers.Nadam(0.000005, name='optimizer')
    model.compile(
        optimizer=optimizer, 
        loss=losses.get(LOSS),
        metrics=[metrics.get(metric) for metric in METRICS]
    )
    return model


# # DeelLabV3+ Model
# model = DeeplabV3Plus(image_size=IMAGE_SIZE, num_classes=NUM_CLASSES)

# model.summary()

# Model Selection/Load

In [ ]:
from IPython.display import Image
m = get_model()
# m = DeeplabV3Plus(IMAGE_SIZE=256, num_classes=1)# DeeplabV3+

# MODEL_VERSION = 'v1_DeepLabV3Plus'
# ESCOLHIDOS PARA PREDICAO

EPOCH           = 0
MODEL_VERSION   = 'final'
CHECK_MODEL_DIR = MODEL_DIR + '/' +MODEL_VERSION
creatDirectory(CHECK_MODEL_DIR)
CHECK_MODEL_DIR = CHECK_MODEL_DIR + '/cp-0'+str(EPOCH)+'.ckpt'
# m.load_weights(CHECK_MODEL_DIR)
print(m.summary())

In [ ]:
import matplotlib.pyplot as plt
import math
import datetime, os

plt.style.use("ggplot")
checkpoint_path = CHECK_MODEL_DIR+"/cp-{epoch:04d}.ckpt"
checkpoint_dir  = os.path.dirname(checkpoint_path)
n = 20 # Number of shards in each polygon.
N = 200 # Total sample size in each polygon.

log_dir = f'~/local_path/apicum_segmentation/output/v{VERSION}/{MODEL_VERSION}'
creatDirectory(log_dir)

tensorboard = tf.keras.callbacks.TensorBoard(log_dir=log_dir+'/log_model',write_images=True)
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,verbose=1, save_weights_only=True, period=5)
img_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=previewClass)


m.save_weights(checkpoint_path.format(epoch=0))

TRAIN_SIZE = 47700 # with data augmentation
# TRAIN_SIZE = 31800
EVAL_SIZE  = 13355
BATCH_SIZE = 12
print(int(TRAIN_SIZE / BATCH_SIZE))

result = m.fit(x=training,
  epochs=100,
  initial_epoch=0, # REMEBER TO CHANGE THIS INITIAL EPOCH PARAM, WHEN OTHER MODEL HAS BEEN LOADED
  steps_per_epoch=int(TRAIN_SIZE / BATCH_SIZE), 
  verbose=1,
  shuffle=True,
  validation_data=evaluation,
  validation_steps=EVAL_SIZE,
  callbacks = [cp_callback,tensorboard, img_callback])


# Grid

In [ ]:
import pygeoj
kernel_buffer   = [256, 256]
image_base_name = 'allPatch_UNET_grid_'
grid            = pygeoj.load('~/local_path/apicum_segmentation/GRID-ALLCALSSES-COL8-19052023.geojson')

# Export mosaics to GDrive

In [46]:
def doExport(out_image_base,index_in, kernel_buffer, roi):
  """Run the image export task. 
  """
  index = index_in
  image = ee.Image('projects/mapbiomas-workspace/TRANSVERSAIS/ZONACOSTEIRA6/mosaic_'+str(index))
  # Export the image, specifying scale and region.
  task = ee.batch.Export.image.toDrive(
    image          = image.select(BANDS).toFloat(),
    description    = out_image_base+'_'+str(index),
    fileNamePrefix = out_image_base+'_'+str(index), 
    folder         = 'mosaics_landsat/'+str(index),
    scale          = 30,
    region         = roi,
    fileFormat     = 'TFRecord',
    formatOptions  = { 
      'patchDimensions': KERNEL_SHAPE,
      'kernelSize': kernel_buffer,
      'compressed': True,
      'maxFileSize': 157286400
    }
  )
  task.start()  

In [ ]:
# Run the export.
for region in grid:
    region_id = int(region.properties['id'])
    if int(region_id) and region.properties['apicum'] == 1:
      print('Region:',int(region_id))
      for y in range(1985, 2021): 
          doExport(image_base_name+str(region_id)+str('_' + MOSAIC_VERSION),y, kernel_buffer, region.geometry.coordinates[0])

# Mosaic prediction

In [15]:
def doPrediction(out_image_base,index_in,region_id, kernel_buffer, region, epochs):
  """Perform inference on exported imagery, upload to Earth Engine.
  """
  from PIL import Image
  import glob
  import rasterio
  from rasterio.plot import show
  import numpy
  import matplotlib.pyplot as plt
  from osgeo import ogr
  from osgeo import gdal
  import os
  import json
  
  
  out_image_base = out_image_base+'_'+str(index_in)
  
  filesList = glob.glob("~/local_path/apicum_segmentation/mosaics_landsat/"+str(index_in)+"/"+out_image_base+"*")
  rasterFolder = f'{OUTPUT_PATH}/{MODEL_VERSION}/{epochs}epochs/classifications_tiff/{index_in}'
  # creatDirectory(rasterFolder)
  rasterURILZW = rasterFolder + '/outimage_'+VERSION+'_'+MODEL_VERSION+'_'+str(region_id)+'_'+str(index_in)+'_'+str(epochs)+'epochs_lzw.tif'

  if os.path.exists(rasterURILZW):
        print('File already predicted \n\n')
        return None

  
  if(len(filesList) == 0):
    print('No files')
    !echo 'ERROR! (No files) grid={y} - {region_id}' >> log.log
    return None
  exportFilesList = [s for s in filesList if out_image_base in s]    
    

  # Get the list of image files and the JSON mixer file.
  imageFilesList = []
  jsonFile = None
  for f in exportFilesList:
    if f.endswith('.tfrecord.gz'):
      imageFilesList.append(f)
    elif f.endswith('.json'):
      jsonFile = f

  # Make sure the files are in the right order.
  imageFilesList.sort() 
  # Load the contents of the mixer file to a JSON object.
  jsonText = open(jsonFile)
  # Get a single string w/ newlines from the IPython.utils.text.SList
  mixer = json.load(jsonText)
  
  patches = mixer['totalPatches']
  cols = int(mixer["patchesPerRow"])
  rows = int(mixer["totalPatches"]/cols)
  

  # Get set up for prediction.
  x_buffer = int(kernel_buffer[0] / 2)
  y_buffer = int(kernel_buffer[1] / 2)
  #print("buffer Size",x_buffer,y_buffer)
  buffered_shape = [
      KERNEL_SHAPE[0] + kernel_buffer[0],
      KERNEL_SHAPE[1] + kernel_buffer[1]]

  imageColumns = [
    tf.io.FixedLenFeature(shape=buffered_shape, dtype=tf.float32) #Tensorflow 2.x
      for k in BANDS
  ]  
    
  imageFeaturesDict = dict(zip(BANDS, imageColumns))
  def parse_image(example_proto):
    return tf.io.parse_single_example(example_proto, imageFeaturesDict) #Tensorflow 2.x

  def toTupleImage(inputs):
    inputsList = [inputs[key] for key in BANDS] #BANDS
    stacked = tf.stack(inputsList, axis=0)
    stacked = tf.transpose(stacked, [1, 2, 0])
    return stacked
  
  # Create a dataset from the TFRecord file(s) in Cloud Storage.
  imageDataset = tf.data.TFRecordDataset(imageFilesList, compression_type='GZIP')
  imageDataset = imageDataset.map(parse_image, num_parallel_calls=4)
  imageDataset = imageDataset.map(toTupleImage).batch(1)
    
    
  # Perform inference.
  predictions = m.predict(imageDataset, steps=patches, verbose=2)
  patchesPerRow  = mixer['patchesPerRow']
  TotalPatches   = mixer['totalPatches']
  patchDimension = mixer['patchDimensions']

  #Manipulating Prediction Numpy Array OUTPUT
  counter       = 1
  rowCounter    = 1
  globalCounter = 0
  finalArray    = numpy.array([])

  rowArray = numpy.array([])
  for raw_record in predictions:
      raw_record = numpy.squeeze(raw_record)
      rows,cols = raw_record.shape      
      raw_record = raw_record[128:384,128:384]
      if rowCounter == 1:
          finalArray = rowArray
      if counter <= patchesPerRow:
          if counter == 1:
              rowArray = raw_record
          else:
              rowArray = numpy.concatenate((rowArray,raw_record), axis = 1)
          counter = counter+1
      else:
          counter = 2
          rowCounter = rowCounter+1
          if numpy.array_equal(finalArray,rowArray):
              finalArray = rowArray
          else:
              finalArray = numpy.concatenate((finalArray,rowArray),axis=0)
          rowArray = raw_record #mod de 73 == 0 new line starts
      globalCounter = globalCounter+1
  finalArray = numpy.concatenate((finalArray,rowArray),axis=0)
  show(Image.fromarray(finalArray))
  rows,cols = finalArray.shape

  driver = gdal.GetDriverByName("GTiff")

  finalArray2  = numpy.array([finalArray])
  rasterFolder = '/~/local_path/apicum_segmentation/output/v'+VERSION+'/'+MODEL_VERSION+'/'+str(epochs)+'epochs/classifications_tiff/'+str(index_in)

  if not os.path.exists(rasterFolder):
    print('lets make the directory')
    os.makedirs(rasterFolder)
  
  rasterURI    = rasterFolder + '/UNET_v'+VERSION+'_'+MODEL_VERSION+'_grid_'+str(region_id)+'_year_'+str(index_in)+'_'+str(epochs)+'epochs.tif'
  rasterURILZW = rasterFolder + '/outimage_'+VERSION+'_'+MODEL_VERSION+'_'+str(region_id)+'_'+str(index_in)+'_'+str(epochs)+'epochs_lzw.tif'
 
  #print(rasterURILZW)
  with rasterio.open(rasterURI,'w',
          driver="GTiff",
          height=rows,
          width=cols,
          count=1,
          dtype="float32",
          crs=mixer["projection"]["crs"],
          transform=mixer["projection"]["affine"]["doubleMatrix"],
          nodata="nan") as dataset:
              dataset.write(finalArray2)
  !gdal_translate -of GTiff -co "COMPRESS=LZW" -co "PREDICTOR=2" -co "TILED=YES" {rasterURI} {rasterURILZW}
  !rm {rasterURI}
    
  print('Writing predictions...')
  out_image_file  = rasterFolder+'/' + out_image_base + '_'+str(epochs)+'epochs.TFRecord'
  out_image_mixer = rasterFolder+'/'+ out_image_base + '_'+str(epochs)+'epochs.json'
  #----------------------------------TFRECORD WRITE ----------------------------
  writer = tf.io.TFRecordWriter(out_image_file)
  patches = 0
  for predictionPatch in predictions:
    #print('Writing patch ' + str(patches) + '...')
    predictionPatch = predictionPatch[
        x_buffer:x_buffer+KERNEL_SIZE, y_buffer:y_buffer+KERNEL_SIZE]

    # Create an example.
    example = tf.train.Example(
      features=tf.train.Features(
        feature={
          'classification': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=predictionPatch.flatten()))
        }
      )
    )
    # Write the example.
    writer.write(example.SerializeToString())
    patches += 1

  writer.close()
  !cp {jsonFile} {out_image_mixer}

In [ ]:
import time
image_base_name = 'allPatch_UNET_grid_'

for y in range(1985, 2024):
    start = time.time()
    print('starting...')
    !echo 'year={y}' >> log.log
    for region in grid:
        region_id = region.properties['id']
        region_id = int(region_id)
        if region.properties['apicum'] == 1 and not (region_id in processed_grids):
            processed_grids.append(region_id)
            print(f"region: {region_id}, year: {y}")
            print('Predicting')
            doPrediction(image_base_name+str(region_id)+str('_' + MOSAIC_VERSION),y,region_id, kernel_buffer, region.geometry.coordinates, EPOCH)
            print('Finish')
            !echo 'grid={y} -{region_id}' >> log.log
    end = time.time()
    print('Prediction Time per year = '+str(end - start))
print('DONE')